In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras.applications import MobileNetV2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [40]:
# Parameters
image_size = (250, 250)  # Match your image size
batch_size = 32

# 1. Load and Preprocess Dataset
train_dir = 'Training'
val_dir = 'Validation'

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'  # Binary classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 940 images belonging to 2 classes.
Found 180 images belonging to 2 classes.


In [41]:
# 2. Load Pre-trained Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(250, 250, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

model = Model(inputs=base_model.input, outputs=x)

C:\Users\User\AppData\Local\Temp\ipykernel_11604\2741492622.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(250, 250, 3))


In [42]:
# 3. Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [53]:
# 4. Train the Model
epochs = 10
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    verbose=1
)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9292 - loss: 0.1549 - val_accuracy: 0.9000 - val_loss: 0.2519
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9424 - loss: 0.1527 - val_accuracy: 0.9167 - val_loss: 0.1761
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9415 - loss: 0.1477 - val_accuracy: 0.9222 - val_loss: 0.1940
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.9481 - loss: 0.1353 - val_accuracy: 0.9167 - val_loss: 0.1984
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9591 - loss: 0.1093 - val_accuracy: 0.9167 - val_loss: 0.2122
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.9674 - loss: 0.0872 - val_accuracy: 0.9167 - val_loss: 0.1834
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.9616 - loss: 0.1029 - val_accuracy: 0.9222 - val_loss: 0.1881
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9691 - loss: 0.1008 - val_accuracy: 0.9333 - val_loss:

In [45]:
# 5. Save and Predict
model.save("smoker_classification_model.h5")